In [22]:
import pymongo
import datetime
import json
import pandas as pd
import re
from textblob import TextBlob

In [23]:
from pymongo import MongoClient
client = MongoClient('localhost', 27017)
db = client['eleicoes_2018_tweet_analisys_db']

In [30]:
tweet_db = db.eleicoes_2018_tweet_analisys_db

In [28]:
list_base_tweet_analisar = []

#"source": {"$regex":"(twitter)"},
#https://docs.mongodb.com/manual/reference/operator/query/not/
#  If you are using Python, you can write the above query with the PyMongo driver and Python’s python:re.compile()

# limpeza Parte 1
#  somente lingua EN
#  source: somente tweets feitos dentro no twitter
#  retirado Retweets (RT ....)
#  pega somente tweets originais, o que nao foi feito em cima de nenhum outro tweet

myquery = { 
            "lang": "en",
            "source": re.compile("(twitter)"),
            "text": { "$not": re.compile("(RT)") },
            "is_quote_status": False
           }

myDoc = tweet_db.find(myquery)

for tweet in myDoc: 
    
    text_tweet = ''
    
    try:
        text_tweet = tweet['extended_tweet']['full_text']
    except:
        text_tweet = tweet['text']
        
    s_regex = "(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+"
    text_tweet = re.sub(s_regex, '', text_tweet)
    
    blob = TextBlob(text_tweet)
    if blob.sentiment.polarity == 0:
        label = 'neutro'
    elif blob.sentiment.polarity > 0.5:
        label = 'positivo'
    else:
        label = 'negativo'
    
    json_tweet = {
        'text': text_tweet,
        'id': tweet['id'],
        'created_at': tweet['created_at'],
        'label': label,
        'blob_sentiment': blob.sentiment
    }
    
    list_base_tweet_analisar.append(json_tweet)
    

# limpeza Parte 2
#  somente lingua EN
#  source: somente tweets feitos dentro no twitter
#  retirado Retweets (RT ....)
#  pega somente tweets com comentario

myquery = { 
            "lang": "en",
            "source": re.compile("(twitter)"),
            "text": { "$not": re.compile("(RT)") },
            "is_quote_status": True
           }

myDoc = tweet_db.find(myquery)

i = 0

for tweet in myDoc: 
    text_tweet = ''
    
    try:
        text_tweet = tweet['extended_tweet']['full_text']
    except:
        text_tweet = tweet['text']
        
    s_regex = "(?:(?:https?|ftp):\/\/)?[\w/\-?=%.]+\.[\w/\-?=%.]+"
    text_tweet = re.sub(s_regex, '', text_tweet)
    
    blob = TextBlob(text_tweet)
    if blob.sentiment.polarity == 0:
        label = 'neutro'
    elif blob.sentiment.polarity > 0.5:
        label = 'positivo'
    else:
        label = 'negativo'
    
    json_tweet = {
        'text': text_tweet,
        'id': tweet['id'],
        'created_at': tweet['created_at'],
        'label': label,
        'blob_sentiment': blob.sentiment
    }
        
    list_base_tweet_analisar.append(json_tweet)

In [29]:
db.drop_collection(db.eleicoes_2018_en_db_clean)
tweet_db = db.eleicoes_2018_en_db_clean
tweet_db_id = tweet_db.insert_many(list_base_tweet_analisar)